# Skip-Gram Data Preparation

**Pipeline: Corpus file → TFRecord training artifacts (triplets and vocabulary)**

Use this notebook to prepare Google 5gram corpora for skip-gram training.

## Pipeline Workflow

1. **Input**: Preprocessed corpus file (e.g., `2019.txt`) in `/vast` storage

2. **Processing**:
    * Convert text corpus to `tf.data.TextLineDataset` object
    * Create vocabulary frequency table for optional downsanmpling in the next step)
    * Generate string triplets 

3. **Output**: TFRecord artifacts in organized subdirectories (e.g., `2019_artifacts/`)

### **Artifact Storage**
The pipeline creates year-specific subdirectories alongside the original text corpora:
<pre>
/vast/edk202/NLP_corpora/.../data/
├── 2018.txt
├── 2019.txt
├── 2020.txt
├── 2018_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
├── 2019_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
└── 2020_artifacts/
    ├── triplets.tfrecord.gz
    └── vocab.tfrecord.gz
</pre>

## Set Up for Data Preparation

In [5]:
# Enable autoreload for development
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Set project root and add src to path
import sys
from pathlib import Path
import os

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [3]:
# Print resource summary
from word2gm_fast.utils.resource_summary import print_resource_summary

print_resource_summary()

E0000 00:00:1752682172.423229  379833 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752682172.433724  379833 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752682173.025866  379833 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752682173.025889  379833 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752682173.025891  379833 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752682173.025892  379833 computation_placer.cc:177] computation placer already registered. Please check linka

<pre>SYSTEM RESOURCE SUMMARY
=============================================
Hostname: cm049.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Partition: short
   Job ID: 63778590
   Node list: cm049

Physical GPU Hardware:
   No physical GPUs allocated to this job

TensorFlow GPU Recognition:
   TensorFlow can access 0 GPU(s)
   Built with CUDA support: True
=============================================</pre>

In [4]:
corpus_path = (
    '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/'
    '6corpus/yearly_files/data/1778.txt'
)

In [29]:
from word2gm_fast.dataprep.corpus_to_dataset import make_dataset

tf_dataset, _ = make_dataset(
    corpus_path,
    cache=True,
    show_summary=True,
    show_properties=True,
    preview_n=10)

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Preview of 10 random retained 5-grams:<br><br>&nbsp;&nbsp;&nbsp;   UNK king george UNK third<br>&nbsp;&nbsp;&nbsp;   UNK fiery dart UNK UNK<br>&nbsp;&nbsp;&nbsp;   UNK west side UNK UNK<br>&nbsp;&nbsp;&nbsp;   UNK one large room UNK<br>&nbsp;&nbsp;&nbsp;   UNK follow word UNK UNK<br>&nbsp;&nbsp;&nbsp;   UNK much even UNK UNK<br>&nbsp;&nbsp;&nbsp;   UNK like one astonish UNK<br>&nbsp;&nbsp;&nbsp;   UNK good heart UNK UNK<br>&nbsp;&nbsp;&nbsp;   UNK mean time UNK order<br>&nbsp;&nbsp;&nbsp;   UNK whisper among UNK servant<br></span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Processed Dataset Properties:<br><br>- Element spec: TensorSpec(shape=(), dtype=tf.string, name=None)<br>- Cardinality: Unknown<br>- Threading: Default settings<br>- Transformations: Cached<br></span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Summary:<br><br>- Retained: 7491<br>- Rejected: 15544<br>- Total: 23035<br></span>

In [30]:
from word2gm_fast.dataprep.dataset_to_frequency import dataset_to_frequency

frequency_table = dataset_to_frequency(tf_dataset)

In [31]:
from word2gm_fast.dataprep.dataset_to_triplets import dataset_to_triplets

triplets_ds, _ = dataset_to_triplets(
    dataset=tf_dataset,
    frequency_table=frequency_table,
    downsample_threshold=1e-5,
    preview_n=10,
    cache=True,
    show_properties=True,
    show_summary=True
)

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Preview of 10 random triplets:<br><br>&nbsp;&nbsp;&nbsp;   (open, frank, vote)<br>&nbsp;&nbsp;&nbsp;   (way, half, beg)<br>&nbsp;&nbsp;&nbsp;   (hall, enter, coach)<br>&nbsp;&nbsp;&nbsp;   (improper, highly, divine)<br>&nbsp;&nbsp;&nbsp;   (life, spare, demonstration)<br>&nbsp;&nbsp;&nbsp;   (writing, account, idea)<br>&nbsp;&nbsp;&nbsp;   (acquaint, well, imputation)<br>&nbsp;&nbsp;&nbsp;   (treatment, consider, intoxication)<br>&nbsp;&nbsp;&nbsp;   (two, three, delight)<br>&nbsp;&nbsp;&nbsp;   (certain, degree, sleep)<br></span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Triplets Dataset Properties:<br><br>- Element spec: TensorSpec(shape=(3,), dtype=tf.string, name=None)<br>- Cardinality: Unknown<br>- Threading: Default settings<br>- Transformations: Mapped, FlatMapped, Cached<br></span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Generated Triplets Summary:<br><br>- Total triplets: 2,284<br>- Unique centers: 1,045<br>- Unique positives: 1,621<br>- Unique negatives: 1,552<br>- Total unique words: 2,354<br></span>

In [32]:
from word2gm_fast.dataprep.index_vocab import triplets_to_integers

integer_triplets, vocab_table, vocab_list, vocab_size, vocab_summary = triplets_to_integers(
    triplets_dataset=triplets_ds,
    frequency_table=frequency_table,
    preview_n=10,
    show_summary=True,
    show_properties=True,
    cache=True
)

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Preview of 10 random integer triplets:<br><br>&nbsp;&nbsp;&nbsp;   (858, 230, 1892)<br>&nbsp;&nbsp;&nbsp;   (10, 100, 1742)<br>&nbsp;&nbsp;&nbsp;   (168, 872, 215)<br>&nbsp;&nbsp;&nbsp;   (575, 779, 1260)<br>&nbsp;&nbsp;&nbsp;   (62, 1188, 284)<br>&nbsp;&nbsp;&nbsp;   (2349, 1810, 495)<br>&nbsp;&nbsp;&nbsp;   (174, 659, 285)<br>&nbsp;&nbsp;&nbsp;   (1652, 1555, 2107)<br>&nbsp;&nbsp;&nbsp;   (36, 464, 1111)<br>&nbsp;&nbsp;&nbsp;   (403, 3, 669)<br></span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Integer Triplets Dataset Properties:<br><br>- Element spec: TensorSpec(shape=(3,), dtype=tf.int32, name=None)<br>- Cardinality: 2284<br>- Threading: Default settings<br>- Transformations: TensorSlices, Cached<br></span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>
Vocabulary Summary:<br><br>- Vocabulary size: 2,355<br>- Index range: 0 to 2,354<br>- UNK token: UNK (index 0)<br>- Sample tokens: would, one, great, make, good<br></span>

In [33]:
from word2gm_fast.io.vocab import write_vocab_to_tfrecord
from word2gm_fast.io.triplets import write_triplets_to_tfrecord
import os

# Define output directory
output_dir = (
    '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/'
    '6corpus/yearly_files/data/1778_artifacts'
)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save vocabulary table (compressed)
vocab_path = f"{output_dir}/vocab.tfrecord"
write_vocab_to_tfrecord(
    vocab_table=vocab_table,
    output_path=vocab_path,
    compress=False
)

# Save integer triplets (compressed)
triplets_path = f"{output_dir}/triplets.tfrecord"
triplet_count = write_triplets_to_tfrecord(
    dataset=integer_triplets,
    output_path=triplets_path,
    compress=False
)

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Writing vocabulary TFRecord to: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1778_artifacts/vocab.tfrecord</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Vocabulary write complete. Words written: 2,355</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Writing TFRecord to: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1778_artifacts/triplets.tfrecord</span>

<span style='font-family: monospace; font-size: 120%; font-weight: normal;'>Write complete. Triplets written: 2,284</span>